## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-10-17-47-49 +0000,bbc,Swedish minister reveals teenage son linked to...,https://www.bbc.com/news/articles/c0k7ek2xlndo
1,2025-07-10-17-36-47 +0000,bbc,UK to return some migrants to France within we...,https://www.bbc.com/news/articles/c4g2edx410wo
2,2025-07-10-17-33-17 +0000,bbc,Six things Trump should know about Liberia aft...,https://www.bbc.com/news/articles/c5y72w17vpwo
3,2025-07-10-17-19-33 +0000,bbc,US cuts visa validity for applicants from thre...,https://www.bbc.com/news/articles/cx23ywg9w82o
4,2025-07-10-17-11-56 +0000,nyt,Rubio Tells Top Russian Diplomat of Trump’s Fr...,https://www.nytimes.com/2025/07/09/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2299/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
16,trump,17
74,tariffs,5
8,uk,5
195,floods,4
196,new,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
10,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,42
49,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...,41
21,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,35
51,2025-07-10-01-07-30 +0000,bbc,Trump threatens Brazil with 50% tariff and dem...,https://www.bbc.com/news/articles/c784ee81y4zo,32
24,2025-07-10-11-30-07 +0000,bbc,Brazil vows to match US tariffs after Trump th...,https://www.bbc.com/news/articles/c62dvyv60z7o,31


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
10,42,2025-07-10-16-08-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
69,24,2025-07-09-19-48-23 +0000,nyt,Former President Yoon Suk Yeol of South Korea ...,https://www.nytimes.com/2025/07/09/world/asia/...
64,19,2025-07-09-20-36-58 +0000,bbc,'Huge wall of water': Texas man stood on meter...,https://www.bbc.com/news/articles/c3358pz41d6o
26,18,2025-07-10-11-18-15 +0000,bbc,Armed police in Romania carry out raids linked...,https://www.bbc.com/news/articles/cp867gjmn25o
65,16,2025-07-09-20-25-00 +0000,wsj,Minutes from the Fed’s June policy meeting tea...,https://www.wsj.com/economy/central-banking/ju...
9,16,2025-07-10-16-08-22 +0000,bbc,Russia's intensifying drone war is spreading f...,https://www.bbc.com/news/articles/c0m8gn7grn2o
51,15,2025-07-10-01-07-30 +0000,bbc,Trump threatens Brazil with 50% tariff and dem...,https://www.bbc.com/news/articles/c784ee81y4zo
40,13,2025-07-10-08-11-18 +0000,nyt,China Surveys Seabeds Where Naval Rivals May O...,https://www.nytimes.com/2025/07/10/world/asia/...
44,12,2025-07-10-05-25-38 +0000,bbc,Reform leader says council first to scrap net ...,https://www.bbc.com/news/articles/c3vde31lwpdo
60,11,2025-07-09-21-57-53 +0000,bbc,Chebbak hat-trick helps Morocco avoid DR Congo...,https://www.bbc.com/sport/football/articles/cx...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
